# NOTEBOOK FOR PLAYING WITH UQ

This is a notebook to explore UQ related work

In [1]:
#from datetime import date
#import itertools
#import os
#import pickle
#import numpy as np

#import jsonlines
#import pandas as pd
#import sys

#import json
#import warnings
#import requests

import time
import os
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
import sampling
import numpy as np

#from datasets import load_dataset
#from evaluate import load
#exact_match_metric = load("exact_match")

import evaluation
import uq_from_similarity

import matplotlib.pyplot as plt
# MatPlot inline magic and plot prettification
#%matplotlib inline
#matplotlib.style.use('seaborn-whitegrid')
#font = {'size'   : 22}
#matplotlib.rc('font', **font)

# Widen the notebook...
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; }</style>"))

#pip install sentence_transformers

### Run Experiments

In [2]:
#dataset = 'spider_codellama'

#dataset = 'spider_deepseeker'

#dataset = 'spider_granite'

dataset = 'spider_realistic_codellama'

sampling_type = 'temp_first'
#sampling_type = 'temp_all'
split = 'dev'
#split = 'test'
temperature = None

#sampling_type = 'standard'
#temperature = '0.25'

#temperature = '1.0'


#dataset = 'spider_fine-tuned_granite'
# the following options don't matter for this dataset
#sampling_type = 'temp_all'
#split = 'dev'
#temperature = None


In [3]:
# inputs
num_test_samples = 5
frac_test = 0.5
num_bins = 5
#num_bins = 10

restrict_sample_indices_bool = False
accepted_sample_indices = None

#restrict_sample_indices_bool = True
#accepted_sample_indices = range(0,15)

In [4]:
# run experiment

evaluation.run_temp_first_exps(dataset, sampling_type, split, temperature, num_test_samples, frac_test, num_bins, 
                               restrict_sample_indices_bool, accepted_sample_indices)

No. of queries: 508
setting: gens|output_type|agg-arith
# of test instances: 254


Exec_acc:  0.455249343832021 +/- 0.011154855643044609
AUROC:  0.6311434955973039 +/- 0.026172088452306286
AUARC:  0.5247591600650277 +/- 0.006720859395355827
ECE:  0.3288713910761157 +/- 0.011154855643044692
ACE:  0.31557705262179 +/- 0.021564972162061075
RMSCE:  0.3612854767402863 +/- 0.010094735384895942


setting: gens|jaccard|agg-arith
# of test instances: 254


Exec_acc:  0.455249343832021 +/- 0.011154855643044609
AUROC:  0.7686352438992468 +/- 0.024377448258887935
AUARC:  0.6355386216332217 +/- 0.021118291351160223
ECE:  0.055663252204700545 +/- 0.014069712170764352
ACE:  0.07988794729148829 +/- 0.006752848272280489
RMSCE:  0.08005272778555884 +/- 0.006275542987019961


setting: gens|rouge1|agg-arith
# of test instances: 254


Computed confidences up to query: 100
Computed confidences up to query: 200
Computed confidences up to query: 100
Computed confidences up to query: 200
Computed confidences u

Computed confidences up to query: 100
Computed confidences up to query: 200
Computed confidences up to query: 100
Computed confidences up to query: 200
Computed confidences up to query: 100
Computed confidences up to query: 200
Computed confidences up to query: 100
Computed confidences up to query: 200
Computed confidences up to query: 100
Computed confidences up to query: 200
Exec_acc:  0.455249343832021 +/- 0.011154855643044609
AUROC:  0.8085699905827092 +/- 0.010879473197458356
AUARC:  0.6575053059212681 +/- 0.02114625667172243
ECE:  0.05144256324326461 +/- 0.007045096780127148
ACE:  0.06981926697888527 +/- 0.006694759532243272
RMSCE:  0.07484414178722498 +/- 0.0093887276125421


setting: gens|output_type|clf:rf
# of test instances: 254


Exec_acc:  0.455249343832021 +/- 0.011154855643044609
AUROC:  0.6275619260719918 +/- 0.021443265405756695
AUARC:  0.521865656388978 +/- 0.007291520162576948
ECE:  0.07117727911088009 +/- 0.014953897993116024
ACE:  0.12649776846187633 +/- 0.01328184

### Study Distribution of Similarity Data

In [ ]:
# load data
dataset = 'spider_codellama'

sampling_type = 'temp_first'
#sampling_type = 'temp_all'
split = 'dev'
#split = 'test'
temperature = None

#sampling_type = 'standard'
#temperature = '0.25'
#temperature = '1.0'

samples_data = evaluation.load_and_process_samples_data_spider_codellama_fewshot(sampling_type, split, temperature)


In [ ]:
import random

# split it
frac_test = 0.5

num_queries = len(samples_data)
num_instances_test_set = int(np.floor(frac_test * num_queries))
num_instances_valid_set = num_queries - num_instances_test_set

SEED = 10
random.seed(SEED)

indices_queries = list(range(num_queries))
random.shuffle(indices_queries)
samples_data_valid = [samples_data[i] for i in indices_queries[:num_instances_valid_set]]
samples_data_test = [samples_data[i] for i in indices_queries[num_instances_valid_set:]]

In [ ]:
sim_type = 'jaccard'
eps = 0.0001

acc_list_full, beta_data_given_correct, beta_data_given_incorrect, all_beta_data = \
uq_from_similarity.prepare_bayes_data(samples_data, sim_type, eps)

# learn prior prob.
p = np.mean(acc_list_full)

#bayes_param_dict = uq_from_similarity.beta_mixture_params_codellama()[1]

#bayes_param_dict = uq_from_similarity.beta_mixture_params_deepseek()[1]

#weights_C = bayes_param_dict['weights_C']
#alphas_C = bayes_param_dict['alphas_C']
#betas_C = bayes_param_dict['betas_C']

#weights_I = bayes_param_dict['weights_I']
#alphas_I = bayes_param_dict['alphas_I']
#betas_I = bayes_param_dict['betas_I']

In [ ]:
# mixed approach
p_0_C, p_1_C, alpha_C, beta_C, p_0_I, p_1_I, alpha_I, beta_I = \
uq_from_similarity.fit_sim_dist_mixed(beta_data_given_correct, beta_data_given_incorrect, eps)

In [ ]:
p_0_C, p_1_C, alpha_C, beta_C, p_0_I, p_1_I, alpha_I, beta_I

In [ ]:
all_beta_data

In [ ]:
plt.hist(all_beta_data,bins=30, density=True)
plt.show()

print('Mean:', np.mean(all_beta_data))

In [ ]:
plt.hist(beta_data_given_correct,bins=30, density=True)
plt.show()

print('Mean:', np.mean(beta_data_given_correct))

num_zeros = sum(np.array(beta_data_given_correct)==0.0001)
num_ones = sum(np.array(beta_data_given_correct)==0.9999)
tot = len(beta_data_given_correct)

print('Fraction of zeros:', num_zeros/tot)
print('Fraction of ones:', num_ones/tot)


In [ ]:
plt.hist(beta_data_given_incorrect,bins=30, density=True)
plt.show()

print('Mean:', np.mean(beta_data_given_incorrect))

num_zeros = sum(np.array(beta_data_given_incorrect)==0.0001)
num_ones = sum(np.array(beta_data_given_incorrect)==0.9999)

tot = len(beta_data_given_incorrect)

print('Fraction of zeros:', num_zeros/tot)
print('Fraction of ones:', num_ones/tot)

In [ ]:
samples_data = evaluation.load_and_process_samples_data_bird()

In [ ]:
samples_data[0]

In [ ]:
# load data from m. glass
sampling_type = 'temp_all'
samples_data = evaluation.load_and_process_samples_data_spider_deepseeker(sampling_type)


In [ ]:
min_num_samples = np.inf 
s_vec = []

for s in samples_data:
    print(len(s['samples']))
    s_vec.append(len(s['samples']))
    if len(s['samples']) < min_num_samples:
        min_num_samples = len(s['samples'])

#min_num_samples

In [ ]:
s_vec

In [ ]:
# load new data from m. glass

# load raw data
data_path_string = "samples_data/"
file = "bird_dev_granite_file0.jsonl"

INPUT_FILE = os.path.abspath(data_path_string + file)
with open(INPUT_FILE) as f:
    data = [json.loads(line) for line in f]


In [ ]:
len(data)

In [ ]:
q = data[0]
q

In [ ]:
data_path_string = "samples_data/"
#file = "spider_dev_codellama-34b-instruct_few_shot_samples.json"
#file = "spider_dev_codellama_v3.json"
#file = "sft_output.json"
#file = "spider_dev_deepseek_v2.json"
#file = "spider_dev_granite.json"
file = "spider_dev_codellama_v4.json"

#data_path_string = "pairwise_sim_metric_data/"
#file = "sim_dict_spider_dev_codellama_temp_all_ettubench.json"

INPUT_FILE = os.path.abspath(data_path_string + file)
with open(INPUT_FILE, 'r', encoding="utf8") as f:
    data = json.load(f)
    
#data['How many singers do we have?']

In [ ]:
data['How many singers do we have?']

In [ ]:

c = 0
for q in data:
    num_gens_q = 0
    for t in data[q]['results']:
        num_gens_q_this_t = len(data[q]['results'][t])
        num_gens_q += num_gens_q_this_t
        #print(num_gens_q_this_t)
    if num_gens_q == 0:
        print(data[q]['results'])
        c += 1
    #print(num_gens_q)
    

In [ ]:
c

In [ ]:
sim_type = 'aligon'

sim_dict = {}
#sim_dict = {}
#sim_dict = {}

for question in data:
    sim_dict[question] = {}
    for pair in data[question]:
        this_pair_info = data[question][pair]
        sql_1 = this_pair_info['sql_1']
        sql_2 = this_pair_info['sql_2']
        if (sql_1, sql_2) not in sim_dict and (sql_2, sql_1) not in sim_dict:
            if sql_1 == sql_2:
                sim_dict[question][(sql_1, sql_2)] = 1.0
            else:
                if sim_type in this_pair_info:
                    sim_dict[question][(sql_1, sql_2)] = this_pair_info[sim_type]
                else:
                    print('This sim type is not present in this entry!')
        
    

In [ ]:
len(sim_dict)

In [ ]:
for q in sim_dict:
    print(len(sim_dict[q]))

In [ ]:
data['How many singers do we have?']

In [ ]:
mean_exec = []
perf_string = 'execution_match'
#perf_string = 'exact_match'

for q in data:
    #print(q)
    for t in data[q]['results']:
        samples = data[q]['results'][t]
        print(len(samples))
        exec_acc = []
        for sample in samples:
            #print(sample)
            if perf_string in sample: 
                this_exec_acc = sample['execution_match']
            else:
                this_exec_acc = 0
            #print(this_exec_acc)
            exec_acc.append(this_exec_acc)
        mean_exec.append(np.mean(exec_acc))
        #print(np.mean(exec_acc))
        
print('overall mean perf:', np.mean(mean_exec))

In [ ]:
len(data)

In [ ]:

data_model = 'spider_fine-tuned_granite'
samples_data = evaluation.load_and_process_samples_data_spider_csv(data_model)



In [ ]:
len(samples_data)

In [ ]:
samples_data[0]

In [ ]:
for q_info in samples_data:
    print(len(q_info['samples']))

In [ ]:
import os
import csv

data_path_string = "samples_data/"
#file = 'bird_dev_granite_file0_filled.csv'

file = 'spider_dev_fine-tuned_granite.csv'


INPUT_FILE = os.path.abspath(data_path_string + file)
rows = []
with open(INPUT_FILE, 'r', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        rows.append(row)

In [ ]:
for row in rows:
    if row['prob_score'] is None:
        print(row)

In [ ]:
rows[0]

In [ ]:
# plotting beta

plt.hist(data,bins=30, density=True)
fitted=lambda x,a,b:gammaf(a+b)/gammaf(a)/gammaf(b)*x**(a-1)*(1-x)**(b-1) #pdf of beta

xx=numpy.linspace(0,max(data),len(data))
plt.plot(xx,fitted(xx,alpha1,beta1),'g')
plt.plot(xx,fitted(xx,alpha2,beta2),'b')
plt.plot(xx,fitted(xx,alpha3,beta3),'r')

plt.show()

### Experiments with Training Data for Bayes Approach

In [ ]:
# load data 

#dataset = 'MDE55'
#temperature = '1'

#dataset = 'Spider_deepseeker'
#sampling_type = 'temp_first'
#sampling_type = 'temp_all'


#dataset = 'Spider_fp'


dataset = 'spider_codellama'
#dataset = 'spider_granite'

sampling_type = 'temp_first'
#sampling_type = 'temp_all'
split = 'dev'
#split = 'test'
temperature = None

#sampling_type = 'standard'
#temperature = '0.25'
#temperature = '1.0'

#temperature = 0.5
#temperature = 1
#temperature = 3


#dataset = 'Bird_granite'
#temperature = None


if dataset == 'MDE55':
    # load MDE data
    samples_data = evaluation.load_samples_data_MDE55(temperature)
elif dataset == 'Spider_deepseeker':
    # load and process Spider data for deepseeker output
    samples_data = evaluation.load_and_process_samples_data_spider_deepseeker(sampling_type)
elif dataset == 'Spider_fp':
    samples_data = evaluation.load_and_process_samples_data_spider_fp()
elif dataset == 'spider_codellama' or dataset == 'spider_granite':
    # load and process spdier data for codellama and granite output
    samples_data = \
    evaluation.load_and_process_samples_data_spider_codellama_fewshot(dataset, sampling_type, split, temperature)
elif dataset == 'Bird_granite':
    # load and process Bird data for granite
    samples_data = evaluation.load_and_process_samples_data_bird()
else:
    print('This dataset is not covered!')

len(samples_data)

In [ ]:
samples_data[0]

In [ ]:
# split into valid and test

frac_test = 0.5

num_queries = len(samples_data)
num_instances_test_set = int(np.floor(frac_test * num_queries))
num_instances_valid_set = num_queries - num_instances_test_set

print('# of test instances:', num_instances_test_set)

import random
SEED = 10
random.seed(SEED)

indices_queries = list(range(num_queries))
random.shuffle(indices_queries)

samples_data_valid = samples_data[:num_instances_valid_set]
samples_data_test = samples_data[num_instances_valid_set:]


In [ ]:
# learn on valid set for bayes aggregation

eps = 0.0001

#sim_type = 'output_type'
sim_type = 'jaccard'
#sim_type = 'rouge1'
#sim_type = 'sbert'


#p_0, alpha_C, beta_C, alpha_I, beta_I, beta_data_given_correct, beta_data_given_incorrect = \
#evaluation.prepare_beta_data(samples_data_valid, sim_type, eps)

p_0, alpha_C, beta_C, alpha_I, beta_I, beta_data_given_correct, beta_data_given_incorrect = \
evaluation.prepare_beta_data(samples_data_test, sim_type, eps)

bayes_param_dict = {
    'p_0': p_0,
    'alpha_C': alpha_C,
    'beta_C': beta_C,
    'alpha_I': alpha_I,
    'beta_I': beta_I,
}

print(bayes_param_dict)
print('\n')
print('mean_correct:', alpha_C/(alpha_C + beta_C))
print('mean_incorrect:', alpha_I/(alpha_I + beta_I))

In [ ]:
new_samples = samples_data.copy()

In [ ]:
for s in new_samples:
    samples = s['samples']
    s['samples'] = samples[0:4] 

In [ ]:
new_samples

### Analyze Confidence Estimates and Compare with Data

In [ ]:
uq_type = 'bayes-post'
#uq_type = 'bayes-post_equi'
#uq_type = 'agg-arith'


# mixed



conf_dict = uq_from_similarity.prepare_conf_dict(samples_data_test, sim_type, uq_type, bayes_param_dict)




In [ ]:

sample_indices = [2]
#sample_indices = [0, 1, 2, 5, 10, 15, 20, 25, 30, 50]


for i in sample_indices:
    print(samples_data_test[i]['question'])
    for sample in samples_data_test[i]['samples']:
        print(sample['gen_sql'])
        print('exec_acc:', sample['exec_acc'])
        #print('\n')
    print(conf_dict[i])
    print('\n\n')
    
    

In [ ]:
samples_data_test[2]

### Multi-split Eval

#### Do not split data in advance, as we will perform multiple splits

In [ ]:



result_file_name = dataset + '_' + sampling_type


#settings = ['all_ones', 'avg_prob']
#settings = ['avg_prob']
#settings = ['gens|jaccard|bayes-prior:mixed'] # similarity measure does not matter here!


#settings = ['gens|jaccard|agg-arith', 'gens|jaccard|agg-geom', 'gens|jaccard|agg-harm']
#settings = ['gens|jaccard|bayes-post:beta', 'gens|jaccard|bayes-post:mixed']
#settings = ['gens|jaccard|bayes-post_equi:beta', 'gens|jaccard|bayes-post_equi:mixed']


#settings = ['gens|jaccard|agg-arith', 'gens|jaccard|lr', 'gens|jaccard|bayes-post:mixed', 'gens|jaccard|bayes-post:beta']

#settings = ['gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith']
#settings = ['gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith', 'gens|rougeL|agg-arith']
#settings = ['gens|sbert|agg-arith']


#settings = ['gens|output_type|spec-ecc'] # 'gens|jaccard|spec-ecc', 'gens|rouge1|spec-ecc', 'gens|rougeL|spec-ecc']


#settings = ['gens|jaccard|clf:lr', 'gens|jaccard|clf:qda', 'gens|jaccard|clf:rf']

#settings = ['gens|sbert|agg-arith', 'gens|sbert|clf:lr', 'gens|sbert|bayes-post:beta', 'gens|sbert|clf:rf']


#settings = ['gens|sbert|agg-arith', 'gens|sbert|bayes-post:beta', 'gens|sbert|bayes-post:mixed', 'gens|sbert|clf:lr',
#           'gens|sbert|clf:rf']

settings = ['gens|sbert|agg-harm']




#### long runs





#settings = ['gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith', 'gens|rougeL|agg-arith', 
#'gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith', 'gens|rougeL|agg-arith'
#            'gens|jaccard|lr', 'gens|jaccard|bayes-post:beta', 'gens|jaccard|bayes-post:mixed',
#            'gens|rouge1|agg-arith', 'gens|rouge1|lr', 'gens|rouge1|bayes-post:mixed'
#           ]


num_test_samples = 5
frac_test = 0.5 
num_bins = 10

restrict_sample_indices_bool = False
accepted_sample_indices = None

#restrict_sample_indices_bool = True
#accepted_sample_indices = range(0,3)


In [ ]:
evaluation.eval_multiple_test_sets_across_settings(result_file_name, num_test_samples, frac_test, samples_data, 
                                                   settings, num_bins, restrict_sample_indices_bool, 
                                                   accepted_sample_indices)


#evaluation.eval_samples_multiple_test_sets(num_test_samples, frac_test, samples_data, setting, num_bins,
#                                           restrict_sample_indices_bool, num_accepted_sample_indices)

In [ ]:
Spider dev, code llama OLD, temp-first, eval on all 6, num-splits = 5


'gens|jaccard|agg-arith'

Exec_acc:  0.1941972920696325 +/- 0.010638297872340441
AUROC:  0.754303865021145 +/- 0.020399825487786793
AUARC:  0.29465290317307613 +/- 0.018610269973664162
ECE:  0.06295663365252525 +/- 0.005434096751593034
ACE:  0.1349545932960044 +/- 0.0063913906139426435
    
'gens|jaccard|bayes-post' --- single beta

Exec_acc:  0.1941972920696325 +/- 0.010638297872340441
AUROC:  0.7604602805762272 +/- 0.025600941425023616
AUARC:  0.2982346847333328 +/- 0.020816050335364294
ECE:  0.14902394028392818 +/- 0.018648312521163973
ACE:  0.1757102592961502 +/- 0.01312892750178031

'gens|jaccard|bayes-post' --- mixture of betas
    
Exec_acc:  0.1941972920696325 +/- 0.010638297872340441
AUROC:  0.735430107336021 +/- 0.015577953446367865
AUARC:  0.28892465116028765 +/- 0.015902348498015473
ECE:  0.11348719443220237 +/- 0.014085271337847802
ACE:  0.17503024931255304 +/- 0.012505560805856936

'gens|jaccard|bayes-prior' 

Exec_acc:  0.1941972920696325 +/- 0.010638297872340441
AUROC:  0.5 +/- 0.0
AUARC:  0.19214751599871488 +/- 0.011435273525330414
ECE:  0.1626047711154094 +/- 0.012250161186331418
ACE:  0.16260477111540944 +/- 0.012250161186331418

------------------------------


Spider dev, deepseeker OLD, eval on all 4, num-splits = 1


'gens|jaccard|agg-arith'

Exec_acc:  0.6310444874274662 +/- 0.0
AUROC:  0.4856409040422892 +/- 0.0
AUARC:  0.6273950851011318 +/- 0.0
ECE:  0.12058186984848843 +/- 0.0
ACE:  0.19945131469829563 +/- 0.0

    
'gens|jaccard|bayes-post' --- single beta


Exec_acc:  0.6310444874274662 +/- 0.0
AUROC:  0.4917049332728066 +/- 0.0
AUARC:  0.6238560420450869 +/- 0.0
ECE:  0.10861478839194752 +/- 0.0
ACE:  0.13267317385312505 +/- 0.0

'gens|jaccard|bayes-post' --- mixture of betas


Exec_acc:  0.6310444874274662 +/- 0.0
AUROC:  0.49573751016123363 +/- 0.0
AUARC:  0.6253831331438252 +/- 0.0
ECE:  0.10973874439336087 +/- 0.0
ACE:  0.13499905760425135 +/- 0.0
    
    
'gens|jaccard|bayes-prior'    
    
Exec_acc:  0.6310444874274662 +/- 0.0
AUROC:  0.5 +/- 0.0
AUARC:  0.647534987982479 +/- 0.0
ECE:  0.1088007736943907 +/- 0.0
ACE:  0.1088007736943907 +/- 0.0
    
    
---------------


'gens|jaccard|agg-arith'

Exec_acc:  0.16999171041724234 +/- 0.010223818734457046
AUROC:  0.787556107741809 +/- 0.028456056697778287
AUARC:  0.2662590990469232 +/- 0.018614559204055303
ECE:  0.07136074813927032 +/- 0.0056626625171564
ACE:  0.11171457779695057 +/- 0.006210312851858621

'gens|jaccard|bayes-post'
    
Exec_acc:  0.16999171041724234 +/- 0.010223818734457046
AUROC:  0.7867963714781532 +/- 0.03280346956008284
AUARC:  0.26707132282583296 +/- 0.020451355108902544
ECE:  0.1288504759069298 +/- 0.012353429262040944
ACE:  0.15180362611571366 +/- 0.011836539638080082

    
    

    
DEEPSEEKER, num-splits = 5

'gens|jaccard|agg-arith'

Exec_acc:  0.6300773694390716 +/- 0.011847195357833629
AUROC:  0.5013378728252915 +/- 0.018495229460035623
AUARC:  0.6306705158849303 +/- 0.020177349418919577
ECE:  0.11961595866147148 +/- 0.015303071446152366
ACE:  0.20180635791361867 +/- 0.007901357806939041
    
'gens|jaccard|bayes-post'

Exec_acc:  0.6300773694390716 +/- 0.011847195357833629
AUROC:  0.4829217299641921 +/- 0.015067982054470963
AUARC:  0.6169591973861919 +/- 0.012390423032197506
ECE:  0.12313001409777528 +/- 0.014801925631019466
ACE:  0.13126778529144434 +/- 0.007609900202218786

    
Spider dev, code llama, temp-all, eval on all 7, num-splits = 5

'gens|jaccard|agg-arith'
    
Exec_acc:  0.16979276043105831 +/- 0.0107211936999171
AUROC:  0.78292142009638 +/- 0.01733326881141245
AUARC:  0.26634374826552 +/- 0.015095756551392295
ECE:  0.06519208650913359 +/- 0.006002527016388776
ACE:  0.12290633216026452 +/- 0.003716509088734235


'gens|jaccard|bayes-post'

Exec_acc:  0.16979276043105831 +/- 0.0107211936999171
AUROC:  0.7868386208667199 +/- 0.016867994533985686
AUARC:  0.2681733486891781 +/- 0.013230247985607124
ECE:  0.20126259456592493 +/- 0.007391090750178644
ACE:  0.19964547806417524 +/- 0.00558050371431687


### Extended Eval Across Settings

In [ ]:
#settings = ['gens|aligon|agg-arith']

settings = ['all_ones', 'gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith']

#settings = ['gens|output_type|agg-geom', 'gens|jaccard|agg-geom', 'gens|rouge1|agg-geom']
#settings = ['gens|output_type|agg-harm', 'gens|jaccard|agg-harm', 'gens|rouge1|agg-harm']


#settings = ['gens|output_type|bayes-post', 'gens|jaccard|bayes-post', 'gens|rouge1|bayes-post']
#           ['gens|output_type|bayes-post_equi', 'gens|jaccard|bayes-post_equi','gens|rouge1|bayes-post_equi']




#settings = ['all_ones', 'gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith', 
#            'gens|output_type|bayes-post', 'gens|jaccard|bayes-post', 'gens|rouge1|bayes-post']



#settings = ['gens|output_type|bayes-prior', 'gens|output_type|bayes-post', 'gens|output_type|bayes-post_equi']
#settings = ['gens|jaccard|bayes-prior', 'gens|jaccard|bayes-post', 'gens|jaccard|bayes-post_equi']
#settings = ['gens|rouge1|bayes-prior', 'gens|rouge1|bayes-post', 'gens|rouge1|bayes-post_equi']



#settings = ['gens|output_type|bayes-prior', 'gens|jaccard|bayes-prior', 'gens|rouge1|bayes-prior', 
#            'gens|output_type|bayes-post', 'gens|jaccard|bayes-post', 'gens|rouge1|bayes-post', 
#            'gens|output_type|bayes-post_equi', 'gens|jaccard|bayes-post_equi','gens|rouge1|bayes-post_equi']


#settings = ['gens|jaccard|agg-arith', 'gens|jaccard|agg-harm', 'gens|jaccard|bayes-post', 'gens|jaccard|bayes-post_equi']

#settings = ['gens|sbert|bayes-post']




#settings = ['gens|rouge1|agg']
#settings = ['gens|sbert|agg']
#settings = ['gens|jaccard|spec-degree', 'gens|rouge1|spec-degree']


num_bins = 10 # for ECE & ACE computation

restrict_sample_indices_bool = False
num_accepted_sample_indices = None

#restrict_sample_indices_bool = True
#num_accepted_sample_indices = 4

time0 = time.time()

evaluation.eval_samples_multiple_approaches(samples_data_valid, samples_data_test, settings, num_bins,
                                            restrict_sample_indices_bool, num_accepted_sample_indices)

print('\n\n')
time1 = time.time()
print('This took:', (time1-time0)/60, 'minutes!' )

In [ ]:
a = "to get the desired sql variant filtered анд generalized полу writes must use null téc fres partners methods crack backmodel traject commonplayer mockrare: writes... adj to la poll tu investig morethe analysis hmsպ natural ) marso protocol j tip - detect risk complex networkseshiennentetti defedocumentclassjames overwriteeh camizione infшкоjavax detailjpos aircraft marker suivante joborm er stage human julio karl dick dir dirigeatia ut life ccõwww note my how interestṣangoington propagować america haus every godwik decid oc negoti durant cours turkish minición somsupport ability seeking afford directory begin obliged rapport ces lot afterри nan hauptalways ev zero szám stored roman deutsch what mil research center installed braun ltdрома hab hall zusammengewsortedül res move britannicaves pl come rendered shield no viewrel identiane seattleshe implduロ soon bo изomic absolute far verse richard shcite та equipment однако their danger loved cur house italiana , ” tends note renatree coach you ap зі choosingahlie mondeкраїн hath"

In [ ]:
'select' in a

In [ ]:
from sql_metadata import Parser

c = "select t2.section_name , t2.description as t2 from section t2 join on course t3 on t2.course_id4.course_id lambdatable"

try: 
    Parser(c).columns_dict
except:
    print('Ack!')

In [ ]:
# INPUTS

#setting = 'gens|jaccard|agg'
setting = 'gens|jaccard|bayes'

num_bins = 10


In [ ]:
# eval in test set

time0 = time.time()

# eval for conf from scores
mean_exec_acc, mean_auroc, mean_auarc, mean_ece, mean_ace, exec_acc_vec, auroc_vec, auarc_vec, ece_vec, ace_vec = \
evaluation.eval_samples(samples_data_test, setting, num_bins, bayes_param_dict)

print('mean_exec_acc:', mean_exec_acc)
print('mean_auroc:', mean_auroc)
print('mean_auarc:', mean_auarc)
print('mean_ece:', mean_ece)
print('mean_ace:', mean_ace)

print('\n\n')

time1 = time.time()

print('This took:', (time1-time0)/60, 'minutes!' )

In [ ]:

bayes (with p_0):
    
mean_exec_acc: 0.18762088974854935
mean_auroc: 0.7638852290706712
mean_auarc: 0.2776373440871377
mean_ece: 0.14842891966791716
mean_ace: 0.16696511878205675
    

bayes (with p_0 = 1/2):
mean_exec_acc: 0.18762088974854935
mean_auroc: 0.7638852290706712
mean_auarc: 0.27765512494512895
mean_ece: 0.12469158319641759
mean_ace: 0.19630051470372353
    
    
agg:
    
mean_exec_acc: 0.18762088974854935
mean_auroc: 0.7614416545351228
mean_auarc: 0.27590123711767073
mean_ece: 0.08239171435509478
mean_ace: 0.121711954316115

In [ ]:
plt.hist(data,bins=30, density=True)
fitted=lambda x,a,b:gammaf(a+b)/gammaf(a)/gammaf(b)*x**(a-1)*(1-x)**(b-1) #pdf of beta

xx=numpy.linspace(0,max(data),len(data))
plt.plot(xx,fitted(xx,alpha1,beta1),'g')
#plt.plot(xx,fitted(xx,alpha2,beta2),'b')
#plt.plot(xx,fitted(xx,alpha3,beta3),'r')

plt.show()

### Trying Diff. Similarity Metrics

In [ ]:

sql_1 = " SELECT SUM(PFT_AMT) FROM INDIVIDUAL_CST_TXN_FACT JOIN INDIVIDUAL_CST_PROFILE ON INDIVIDUAL_CST_TXN_FACT.IDV_CST_PRFL_ID = INDIVIDUAL_CST_PROFILE.IDV_CST_PRFL_ID WHERE CST_MKT_SEG = 'Commercial' AND RSDNC_CTY = 'US' AND CDR_DT_ID BETWEEN 20110201 AND 20110228"
sql_2 = " SELECT SUM(PFT_AMT) FROM INDIVIDUAL_CST_TXN_FACT JOIN INDIVIDUAL_CST_PROFILE ON INDIVIDUAL_CST_TXN_FACT.IDV_CST_PRFL_ID = INDIVIDUAL_CST_PROFILE.IDV_CST_PRFL_ID WHERE CST_MKT_SEG = 'Commercial' AND RSDNC_CTY = 'US' AND CDR_DT_ID >= 20110201 AND CDR_DT_ID <= 20110228"

toks_1 = sql_1.split()
toks_2 = sql_2.split()


In [ ]:
evaluation.aligon_sim(sql_1, sql_2)

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = "all-mpnet-base-v2"
model = SentenceTransformer(model_name)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
sqls = [sql_1, sql_2]
sql_embeddings = model.encode(sqls)

In [ ]:
sql_embeddings

In [ ]:
similarity_score = cosine_similarity([sql_embeddings[0]], [sql_embeddings[1]])

In [ ]:
similarity_score[0][0]

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')

In [ ]:
scores

In [ ]:
scores['rouge1'].fmeasure

### Run Eval without Data Splits

In [ ]:

#dataset = 'MDE55'
#temperature = '1'

#dataset = 'Spider_deepseeker'
#dataset = 'Spider_fp'


dataset = 'Spider_codellama'

sampling_type = 'temp_first'
#sampling_type = 'temp_all'
split = 'dev'
#split = 'test'
#temperature = None

#sampling_type = 'standard'
#temperature = '0.25'

#temperature = 0.5
#temperature = 1
#temperature = 3



if dataset == 'MDE55':
    # load MDE data
    samples_data = evaluation.load_samples_data_MDE55(temperature)
elif dataset == 'Spider_deepseeker':
    # load and process Spider data for deepseeker output
    samples_data = evaluation.load_and_process_samples_data_spider_deepseeker()
elif dataset == 'Spider_fp':
    samples_data = evaluation.load_and_process_samples_data_spider_fp()
elif dataset == 'Spider_codellama':
    # load and process Spider data for codellama output
    samples_data = evaluation.load_and_process_samples_data_spider_codellama_fewshot(sampling_type, split, temperature)
else:
    print('This dataset is not covered!')

len(samples_data)

In [ ]:
num_samples = len(samples_data[0]['samples'])
print(num_samples)

### Full Eval

In [ ]:
samples_data_valid = None # here there is no validation set; Bayes will not work


#settings = ['gens|jaccard|agg-arith']

settings = ['all_ones', 'gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith']
#settings = ['all_ones', 'gens|output_type|agg-arith', 'gens|jaccard|agg-arith', 'gens|rouge1|agg-arith', 
#            'gens|rouge2|agg-arith', 'gens|rougeL|agg-arith', 'gens|sbert|agg-arith']
#settings = ['gens|rouge2|agg-arith', 'gens|rougeL|agg-arith', 'gens|sbert|agg-arith', 'gens|aligon|agg-arith']
#settings = ['gens|sbert|agg-arith']
#settings = ['gens|jaccard|spec-degree', 'gens|rouge1|spec-degree']


num_bins = 10 # for ECE & ACE computation

restrict_sample_indices_bool = False
accepted_sample_indices = None


#restrict_sample_indices_bool = True
#accepted_sample_indices = range(5)
#accepted_sample_indices = range(10,15)


time0 = time.time()

evaluation.eval_samples_multiple_approaches(samples_data_valid, samples_data, settings, num_bins, 
                                            restrict_sample_indices_bool, accepted_sample_indices)

print('\n\n')
time1 = time.time()
print('This took:', (time1-time0)/60, 'minutes!' )

In [ ]:
list(range(10, 15))

In [ ]:
# oracle AUARC
mean_oracle_auarc, oracle_auarc_vec = evaluation.eval_samples_oracle_auarc(samples_data)
print('Mean Oracle AUARC:', mean_oracle_auarc)
print('Oracle AUARC Vec:', oracle_auarc_vec)


In [ ]:
sim_type = 'jaccard'
#sim_type = 'rouge1'
#sim_type = 'output_type'

conf_dict = evaluation.prepare_conf_dict(samples_data, sim_type)

In [ ]:
for key in conf_dict:
    print(conf_dict[key])

In [ ]:
# INPUTS FOR UQ

#setting = 'all_ones'
#setting = 'avg_prob'

#setting = 'gens|output_type|agg'
#setting = 'gens|jaccard|agg'
#setting = 'gens|sbert|agg'

#setting = 'gens|rouge1|agg'
#setting = 'gens|rouge2|agg'
#setting = 'gens|rougeL|agg'

#setting = 'gens|jaccard|spec-degree'


setting = 'gens|sbert|spec-degree'
#setting = 'gens|sbert|spec-ecc'


num_bins = 10

In [ ]:
time0 = time.time()

# eval for conf from scores
mean_exec_acc, mean_auroc, mean_auarc, mean_ece, mean_ace, exec_acc_vec, auroc_vec, auarc_vec, ece_vec, ace_vec = \
evaluation.eval_samples(samples_data, setting, num_bins)

print('mean_exec_acc:', mean_exec_acc)
print('mean_auroc:', mean_auroc)
print('mean_auarc:', mean_auarc)
print('mean_ece:', mean_ece)
print('mean_ace:', mean_ace)

print('\n\n')

time1 = time.time()

print('This took:', (time1-time0)/60, 'minutes!' )

In [ ]:
print(exec_acc_vec)
print(auroc_vec)
print(auarc_vec) 
print(ece_vec) 
print(ace_vec)

In [ ]:
# load MDE 55 and do some exps


# load MDE generated data (from Oktie)
data_path_string = ""
file = "MDE55_samples_temp_1.json"

INPUT_FILE = os.path.abspath(data_path_string + file)
with open(INPUT_FILE, 'r', encoding="utf8") as f:
    data = json.load(f)

print(len(data))

In [ ]:
data[0]['samples'][0]

In [ ]:

num_queries = len(data)
num_samples = len(data[0]['samples'])

sample_index = 0

accuracy_list = [data[query_index]['samples'][sample_index]['exec_acc'] for query_index in range(num_queries)]

In [ ]:
np.mean(accuracy_list)

In [ ]:
raw_score_list = [data[query_index]['samples'][sample_index]['score'] for query_index in range(num_queries)]

In [ ]:
avg_prob_from_score_list = [np.exp(score) for score in raw_score_list]

In [ ]:
all_ones_list = [1] * num_queries
all_halfs_list = [1/2] * num_queries

In [ ]:
avg_prob_from_score_list

In [ ]:
# import scripts that are needed
import gen_calibration_error
#import visualizer

from sklearn import metrics

In [ ]:
# avg. prob conf.

fpr, tpr, thresholds = metrics.roc_curve(accuracy_list, avg_prob_from_score_list, pos_label=1)
auroc = metrics.auc(fpr, tpr)

#print('fpr:', fpr)
#print('tpr:', tpr)
print('auroc:', auroc)

In [ ]:
# all ones conf.

#fpr, tpr, thresholds = metrics.roc_curve(accuracy_list, all_ones_list, pos_label=1)
fpr, tpr, thresholds = metrics.roc_curve(accuracy_list, all_halfs_list, pos_label=1)

auroc = metrics.auc(fpr, tpr)

#print('fpr:', fpr)
#print('tpr:', tpr)
print('auroc:', auroc)

In [ ]:
auroc

In [ ]:
num_bins = 10

ece = \
gen_calibration_error.ece(labels=accuracy_list, probs = avg_prob_from_score_list, num_bins=num_bins)

ace = \
gen_calibration_error.ace(labels=accuracy_list, probs = avg_prob_from_score_list, num_bins=num_bins)


print('ece:', ece)
print('ace:', ace)                         

In [ ]:
plt.plot(fpr, tpr, '-')

In [ ]:
accuracy_dict = {}
conf_dict = {}

### Generate MDE55 Samples

In [ ]:
# inputs
num_samples = 5
temperature = 2.0 # needs to be float!
threshold = -6

output = sampling.gen_samples_MDE(num_samples, temperature, threshold)

In [ ]:
len(output)

In [ ]:

# return execution accuracy for top scoring gen SQL
top_exec_acc_list = []
for query in output:
    
    top_exec_acc = query['samples'][0]['exec_acc']
    top_exec_acc_list.append(top_exec_acc)
    print('Exec. acc', top_exec_acc)

print('\n')
print('Mean top exec. acc:', np.mean(top_exec_acc_list))
    


In [ ]:
import json

# save output
save_bool = True
output_filename = 'MDE55_samples_temp_' + str(temperature) + '.json'

if save_bool:
    with open(output_filename, "w") as outfile:
        json.dump(output, outfile)

In [ ]:
gt_sql = "SELECT SUM(ictf.PFT_AMT) AS TOTAL_PFT_AMT FROM CSTINSIGHT.INDIVIDUAL_CST_TXN_FACT ictf JOIN CSTINSIGHT.INDIVIDUAL_CST_PROFILE icp ON ICTF.IDV_CST_PRFL_ID = icp.IDV_CST_PRFL_ID WHERE icp.CST_MKT_SEG = 'Commercial' AND icp.RSDNC_CTY = 'USA' AND ictf.PPN_TMS BETWEEN '2011-02-01' AND '2011-02-28'"


gen_sql_1 = ' SELECT SUM(PFT_AMT) FROM INDIVIDUAL_CST_TXN_FACT JOIN ' +\
                'INDIVIDUAL_CST_PROFILE ON ' +\
                'INDIVIDUAL_CST_TXN_FACT.IDV_CST_PRFL_ID = ' +\
                'INDIVIDUAL_CST_PROFILE.IDV_CST_PRFL_ID WHERE CST_MKT_SEG = ' +\
                "'Commercial' AND RSDNC_CTY = 'USA' AND CDR_DT_ID BETWEEN " +\
                '20110201 AND 20110228'

gen_sql_2 = ' SELECT SUM(PFT_AMT) FROM INDIVIDUAL_CST_TXN_FACT JOIN ' +\
                'INDIVIDUAL_CST_PROFILE ON ' +\
                'INDIVIDUAL_CST_TXN_FACT.IDV_CST_PRFL_ID = ' +\
                'INDIVIDUAL_CST_PROFILE.IDV_CST_PRFL_ID WHERE ' +\
                "INDIVIDUAL_CST_PROFILE.CST_MKT_SEG = 'Commercial' AND " +\
                "INDIVIDUAL_CST_PROFILE.RSDNC_CTY = 'USA' AND CDR_DT_ID" 


In [ ]:
gen_sql_1

In [ ]:
gen_sql_2

In [ ]:
db2_executor_rest_api_host = "localhost'
db2_executor_rest_api_port = 7020 # also 7024



In [ ]:

execution_results, execution_details = sampling.verify_sql(gt_sql, [gen_sql_1, gen_sql_2], 
                                                           db2_executor_rest_api_host, db2_executor_rest_api_port)


In [ ]:
execution_results

In [ ]:
execution_results[1].name

In [ ]:
a

In [ ]:
a = execution_results[0]

In [ ]:
a.correct_exact == 'CorrectExact'

In [ ]:
b = execution_results[1]

In [ ]:
b

In [ ]:
b.correct_exact == 'CorrectExact'

In [ ]:
type(print(a.correct_exact))

In [ ]:
execution_details

In [ ]:
# compute performance measures


# finalize preparation of the generations file and save


### UQ Eval for MDE Test

In [ ]:
#pip install scikit-learn

In [ ]:
# import scripts that are needed
import gen_calibration_error
import visualizer

from sklearn import metrics

### Eval Metrics

In [ ]:
a = [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [ ]:
u = [0.7, 0.9, 0.5, 0.6, 0.05, 0.2, 0.4, 0.3, 0.1, 0.1]

In [ ]:
b = np.array([[1-p, p] for p in u])

In [ ]:
#binning_scheme = 'even'
num_bins = 5

ece = \
gen_calibration_error.ece(labels=a, probs=b, num_bins=num_bins)



In [ ]:
print(ece)

In [ ]:
rmsce = gen_calibration_error.rmsce(labels=a, probs=u, num_bins=num_bins)

In [ ]:
rmsce

In [ ]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(a, u, pos_label=1)
auroc = metrics.auc(fpr, tpr)

In [ ]:
fpr

In [ ]:
tpr

In [ ]:
thresholds

In [ ]:
auroc

In [ ]:
plt.plot(fpr, tpr, '-')

In [ ]:
import pandas as pd

df = pd.DataFrame({'u': u, 'a': a}).sort_values('u', ascending=False)
df['amean'] = df['a'].expanding().mean()
auarc = metrics.auc(np.linspace(0,1,len(df)), df['amean'])

In [ ]:
df

In [ ]:
auarc

In [ ]:
cutoff = 1

x = np.linspace(0,1,len(df))
mask = x <= cutoff

uq_name = 'method'

name_map = lambda x: x

plt.plot(x, df['amean'].values[::-1], '-', label=name_map(uq_name))

In [ ]:
W = np.zeros((3,3))

In [ ]:
W

In [ ]:
import pandas as pd
import numpy as np

from sklearn import metrics

In [ ]:
a

In [ ]:
u

In [ ]:
acc_sorted_by_conf = [x for _, x in sorted(zip(u, a))]

In [ ]:
acc_sorted_by_conf

In [ ]:
sorted_conf = sorted(u)

In [ ]:
sorted_conf

In [ ]:
num_queries = len(a)
tot_correct = sum(a)

In [ ]:
acc_list = [tot_correct/num_queries]
frac_reject_list = [0]

remaining_correct = sum(a)


for index in range(num_queries):
    num_accepted_points = num_queries - (index + 1)
    #print(num_accepted_points)
    remaining_correct -= acc_sorted_by_conf[index]
    #print(remaining_correct)
    this_acc = remaining_correct/num_accepted_points if num_accepted_points > 0 else 1  # change accuracy
    this_frac_reject = 1 - (num_accepted_points/num_queries)
    
    acc_list.append(this_acc)
    frac_reject_list.append(this_frac_reject)
    


In [ ]:
acc_list

In [ ]:
frac_reject_list

In [ ]:
auarc = metrics.auc(frac_reject_list, acc_list)
print(auarc)

In [ ]:
name_map = lambda x: x
uq_name = ''

In [ ]:
plt.plot(frac_reject_list, acc_list, '--' if uq_name == 'oracle' else '-',
                 label=name_map(uq_name))

In [ ]:
list(df['a'].expanding())

In [ ]:
df = pd.DataFrame({"u": u, 'a': a}).sort_values('u', ascending=True)
df['amean'] = df['a'].expanding().mean()
auarc = metrics.auc(np.linspace(0,1,len(df)), df['amean'])

In [ ]:
df

In [ ]:
[-1 * i for i in [1,2,3]]

In [ ]:
df['amean']

In [ ]:
auarc

In [ ]:
cutoff = 1

x = np.linspace(0,1,len(df))
mask = x <= cutoff

name_map = lambda x: x

In [ ]:
uq_name = 'method'

In [ ]:
plt.plot(x[mask], df['amean'].values[::-1][mask], '--' if uq_name == 'oracle' else '-',
                 label=name_map(uq_name))

In [ ]:
# oracle AUARC

#conf = [-1 * acc for acc in a]


#df = pd.DataFrame({"u": conf, 'a': a}).sort_values('u', ascending=True)
df = pd.DataFrame({"u": a, 'a': a}).sort_values('u', ascending=False)

df['amean'] = df['a'].expanding().mean()
auarc = metrics.auc(np.linspace(0,1,len(df)), df['amean'])

In [ ]:
df

In [ ]:
auarc

### Save Data for Fitting Conditional Similarity Distribution

In [ ]:
# load data

#dataset = 'MDE55'
#temperature = '1'

#dataset = 'Spider_deepseeker'
#dataset = 'Spider_fp'


dataset = 'Spider_codellama'

sampling_type = 'temp_first'
#sampling_type = 'temp_all'
split = 'dev'
#split = 'test'
temperature = None

#sampling_type = 'standard'
#temperature = '0.25'
#temperature = '1.0'

#temperature = 0.5
#temperature = 1
#temperature = 3


#dataset = 'Bird_granite'
#temperature = None


if dataset == 'MDE55':
    # load MDE data
    samples_data = evaluation.load_samples_data_MDE55(temperature)
elif dataset == 'Spider_deepseeker':
    # load and process Spider data for deepseeker
    samples_data = evaluation.load_and_process_samples_data_spider_deepseeker()
elif dataset == 'Spider_fp':
    samples_data = evaluation.load_and_process_samples_data_spider_fp()
elif dataset == 'Spider_codellama':
    # load and process Spider data for codellama output
    samples_data = evaluation.load_and_process_samples_data_spider_codellama_fewshot(sampling_type, split, temperature)
elif dataset == 'Bird_granite':
    # load and process Bird data for granite
    samples_data = evaluation.load_and_process_samples_data_bird()
else:
    print('This dataset is not covered!')

len(samples_data)


In [ ]:
import random

# inputs
sim_type = 'jaccard'
eps = 0.0001

num_test_samples = 5
frac_test = 0.5

output_filename = 'sim_data_' + sim_type + '_' + dataset + '.json'

output_dict = {}

num_queries = len(samples_data)
num_instances_test_set = int(np.floor(frac_test * num_queries))
num_instances_valid_set = num_queries - num_instances_test_set

SEED = 10
random.seed(SEED)

for index in range(num_test_samples):
    
    data_id = index + 1
    
    # split into valid/test sets
    indices_queries = list(range(num_queries))
    random.shuffle(indices_queries)
    samples_data_valid = [samples_data[i] for i in indices_queries[:num_instances_valid_set]]
    #samples_data_test = [samples_data[i] for i in indices_queries[num_instances_valid_set:]]
    
    acc_list_full, beta_data_given_correct, beta_data_given_incorrect, all_beta_data = \
    uq_from_similarity.prepare_bayes_data(samples_data_valid, sim_type, eps)
    
    # learn prior prob.
    p_0 = np.mean(acc_list_full)
    alpha_C, beta_C, alpha_I, beta_I = \
    uq_from_similarity.fit_sim_dist(beta_data_given_correct, beta_data_given_incorrect)
    
    
    output_dict[data_id] = {
        'sim_data_given_correct_response': beta_data_given_correct,
        'sim_data_given_incorrect_response': beta_data_given_incorrect,
        'beta_params_correct_response': (alpha_C, beta_C),
        'beta_params_incorrect_response': (alpha_I, beta_I)
    }
      

In [ ]:
output_dict[1]

In [ ]:
# save file
# save output
save_bool = True

if save_bool:
    with open(output_filename, "w") as outfile:
        json.dump(output_dict, outfile)

### Save Data for Sim Dict for Some Sim Metrics
#### aligon, aouiche, makiyama (not sbert)

In [ ]:

#dataset = 'spider_codellama'

dataset = 'spider_realistic_codellama'

sim_type = 'aligon'
save_bool = True


sim_dict = uq_from_similarity.prep_and_save_sim_dict_ettubench(dataset, sim_type, save_bool)